In [75]:
import polars as pl
import numpy as np
import geopandas as gpd
from pathlib import Path
import pyarrow
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline

PARENT_FOLDER = Path().resolve().parent
CURRENT_DIR = Path().resolve()

In [87]:
file_path = CURRENT_DIR.parent / "Data" / "Travel" / "US Commute Flows by Residence County.csv"
us_commute_flows = pl.read_csv(file_path, encoding='utf8')

file_path = CURRENT_DIR.parent / "Data" / "Travel" / "UK Commute Flows by Area Code.csv"
uk_commute_flows = pl.read_csv(file_path, encoding='utf8')

In [95]:
uk_commute_flows

Residence_Area_Code,Residence_Area_Name,Workplace_Area_Code,Workplace_Area_Name,Count
str,str,str,str,i32
"""E06000001""","""Hartlepool""","""E06000001""","""Hartlepool""",14513
"""E06000001""","""Hartlepool""","""E06000002""","""Middlesbrough""",1500
"""E06000001""","""Hartlepool""","""E06000003""","""Redcar and Cleveland""",671
"""E06000001""","""Hartlepool""","""E06000004""","""Stockton-on-Tees""",4240
"""E06000001""","""Hartlepool""","""E06000005""","""Darlington""",398
…,…,…,…,…
"""W06000024""","""Merthyr Tydfil""","""W06000020""","""Torfaen""",228
"""W06000024""","""Merthyr Tydfil""","""W06000021""","""Monmouthshire""",254
"""W06000024""","""Merthyr Tydfil""","""W06000022""","""Newport""",242


In [88]:
uk_commute_flows = uk_commute_flows.with_columns(
    pl.col("Count").str.replace_all(r"[^0-9]", "").cast(pl.Int32).alias("Count")
)
uk_commute_flows

us_commute_flows = us_commute_flows.with_columns(
    pl.col("Workers_In_Commuting_Flow").str.replace_all(r"[^0-9]", "").cast(pl.Int32),
    pl.col("Margin_of_Error").str.replace_all(r"[^0-9]", "").cast(pl.Int32)
)

In [96]:
uk_commute_flows.group_by("Workplace_Area_Name").agg(
    pl.col("Count").sum().alias("Total_Trips_To_Workplace")
).sort("Total_Trips_To_Workplace", descending=True)

Workplace_Area_Name,Total_Trips_To_Workplace
str,i32
"""Birmingham""",287033
"""Leeds""",222151
"""Westminster""",199443
"""Manchester""",184962
"""Sheffield""",153589
…,…
"""Mid Ulster""",37
"""East Dunbartonshire""",35
"""Clackmannanshire""",34


In [94]:
us_commute_flows.group_by("Workplace_County_Name").agg(
    pl.col("Workers_In_Commuting_Flow").sum()
).sort("Workers_In_Commuting_Flow", descending=True)

Workplace_County_Name,Workers_In_Commuting_Flow
str,i32
"""Los Angeles County""",4906906
"""Orange County""",2806483
"""Cook County""",2684000
"""Harris County""",2520499
"""New York County""",2494885
…,…
"""Treasure County""",234
"""Petroleum County""",229
"""Issaquena County""",203
